In [4]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import os
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd

In [5]:
load_dotenv()  # This loads the variables from .env

True

In [8]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [28]:
account_info = pd.read_csv("/Users/lroberts/Downloads/Bulk Uploader Reference - Sheet1.csv")
account_info.head()

,Account name,Ad Account ID,Budget,utm_Next Month Free,utm_Free Class,utm_Quit Free,utm_Presales,utm_Half Off First Month,utm_2 For 20,utm_2 Free,utm_Free Week,utm_Free Month,utm_Transformation Challenge,utm_$24 Month
0,"Orangetheory Bedminster, NJ (Regional)",284726299130471.00,"$1,000.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
1,"Orangetheory Madison, AL",633860747073903.00,"$1,500.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
2,"Orangetheory Starkville, MS",348958262391635.00,$700.00,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
3,"Orangetheory Bel-Air, MD",477143946095880.00,"$2,500.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
4,"Orangetheory Jersey City, NJ",1229642834071405.00,$750.00,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...


In [29]:
account_info.dropna(inplace=True)

In [30]:
account_info['Ad Account ID'] = account_info['Ad Account ID'].apply(lambda x: f'act_{int(x)}')

In [31]:
account_info.head()

,Account name,Ad Account ID,Budget,utm_Next Month Free,utm_Free Class,utm_Quit Free,utm_Presales,utm_Half Off First Month,utm_2 For 20,utm_2 Free,utm_Free Week,utm_Free Month,utm_Transformation Challenge,utm_$24 Month
0,"Orangetheory Bedminster, NJ (Regional)",act_284726299130471,"$1,000.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
1,"Orangetheory Madison, AL",act_633860747073903,"$1,500.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
2,"Orangetheory Starkville, MS",act_348958262391635,$700.00,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
3,"Orangetheory Bel-Air, MD",act_477143946095880,"$2,500.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...
4,"Orangetheory Jersey City, NJ",act_1229642834071405,$750.00,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...


In [27]:
account_ids[0:5]

['act_284726299130471',
 'act_633860747073903',
 'act_348958262391635',
 'act_477143946095880',
 'act_1229642834071405']

In [10]:
def get_audiences(ad_account_id):
    FacebookAdsApi.init(access_token=os.getenv('FACEBOOK_ACCESS_TOKEN'),
                        app_id=os.getenv('FACEBOOK_APP_ID'),
                        app_secret=os.getenv('FACEBOOK_APP_SECRET'))

    ad_account = AdAccount(ad_account_id)

    try:
        saved_audiences = ad_account.get_saved_audiences(fields=[
            'id',
            'name',
            'time_updated',
            # Add other fields as needed
        ])
        # Convert Cursor to list of dictionaries
        audience_list = [audience.export_all_data() for audience in saved_audiences]
        return audience_list
    except Exception as e:
        print(f'An error occurred: {e}')


In [11]:
# Replace 'act_453485805236160' with your actual ad account ID
audiences = get_audiences('act_453485805236160')
for audience in audiences:
    print(audience)

{'id': '23843860520770626', 'name': 'Broad', 'time_updated': '2022-04-15T16:54:31+0000'}
{'id': '23843863636030626', 'name': 'W/E/P/E', 'time_updated': '2022-04-15T16:52:51+0000'}
{'id': '23843866198050626', 'name': 'F/C', 'time_updated': '2022-04-15T16:53:38+0000'}
{'id': '23843866199350626', 'name': 'AI - Members', 'time_updated': '2021-03-22T16:41:27+0000'}
{'id': '23846903082240626', 'name': 'F/C (age cap)', 'time_updated': '2022-04-15T16:55:01+0000'}
{'id': '23846903083510626', 'name': 'Broad (age cap)', 'time_updated': '2022-04-15T16:55:51+0000'}
{'id': '23849433938990626', 'name': "Target (Test Feb '22)", 'time_updated': '2022-02-10T18:47:27+0000'}
{'id': '23849433941100626', 'name': "AI - CR (Test Feb '22)", 'time_updated': '2022-04-15T16:52:12+0000'}
{'id': '23849433942630626', 'name': "F/C (Test Feb '22)", 'time_updated': '2022-04-15T16:47:45+0000'}
{'id': '23849433943550626', 'name': "Broad (Test Feb '22)", 'time_updated': '2022-04-15T16:44:47+0000'}
{'id': '2384985655504062

In [17]:
def find_most_recent_audience(audiences, keywords):

    # Filter audiences based on keywords
    filtered_audiences = [aud for aud in audiences if any(keyword.lower() in aud.get('name', '').lower() for keyword in keywords)]

    if not filtered_audiences:
        return None, f"No audience found containing the keywords '{keywords}'."

    # Find the most recent audience
    most_recent_audience = max(filtered_audiences, key=lambda aud: datetime.strptime(aud.get('time_updated', '1900-01-01T00:00:00+0000'), '%Y-%m-%dT%H:%M:%S%z'))

    return most_recent_audience, None


In [18]:
# Simplified keyword lists
broad_keywords = ['broad']
wep_keywords = ['w/e/p/e', 'wep']
reach_keywords = ['reach', 'target area']

broad_audience, broad_error = find_most_recent_audience(audiences, broad_keywords)
wep_audience, wep_error = find_most_recent_audience(audiences, wep_keywords)
reach_audience, reach_error = find_most_recent_audience(audiences, reach_keywords)

In [19]:
for aud, error, category in zip([broad_audience, wep_audience, reach_audience], [broad_error, wep_error, reach_error], ['Broad', 'W/E/P/E', 'Reach or Target Area']):
    if error:
        print(error)
    else:
        print(f"Most recent '{category}' audience: ID - {aud['id']}, Name - {aud['name']}")


Most recent 'Broad' audience: ID - 23851961174370626, Name - Broad 18+ v3
Most recent 'W/E/P/E' audience: ID - 23851961172890626, Name - W/E/P/E + AI v3
Most recent 'Reach or Target Area' audience: ID - 23851891868640626, Name - Reach v3
